# VNA V3

Want to be able to live plot both S-parameters and the data-read in at once. Also cleans up the imports.

In [1]:
import sys
import os
import importlib #
sys.path.append(os.path.abspath('../scripts'))

#widgets and display
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output
import pprint
from ipywidgets import Layout

#Plotting 
from bqplot import pyplot as plt
import threading
from threading import Thread

#caculations
import numpy as np 
from scipy import signal
import time
import math

# Import functions from custom scripts
import sig_source, sig_source_ui, data_proccess, adl5960
importlib.reload(sig_source)
importlib.reload(sig_source_ui)
importlib.reload(data_proccess)
importlib.reload(adl5960)
from sig_source import SigSource
from sig_source_ui import SignalSourceUI
from data_proccess import VNAfunc
from adl5960 import ADL5960

In [2]:
#Import Pynq
from pynq import PL, allocate, Overlay, GPIO
import xrfdc

### Upload Code to RFSOC

In [3]:
rfsoc_button = widgets.Button(description="Print RFSOC Code Lines")

rfosc_out = Output()
def print_rfsoc(func):
    with rfosc_out:
        rfosc_out.clear_output
        try: 
            pprint.pprint(ol.ip_dict)
        except Exception as e:
            print(f"Error: {e}")


PL.reset() #important fixes caching issues which have popped up.
ol = Overlay('../design_1.bit')  #locate/point to the bit file
dma_interface = ol.axi_dma_0
print(ol.ip_dict.keys())

rf = ol.usp_rf_data_converter_0

rfsoc_button.on_click(print_rfsoc)
display(widgets.VBox([widgets.Label(value="Print RFSOC Output"), rfsoc_button, rfosc_out])) #Display button only after the code is uploaded 

dict_keys(['axi_dma_0', 'spi_adl_1', 'spi_lmx_0', 'spi_adl_0', 'usp_rf_data_converter_0', 'zynq_ultra_ps_e_0'])


In [20]:
print(rf.adc_tiles)

for adc_tile in rf.adc_tiles:
    for block in adc_tile.blocks[:2]:
        for threshold in block.thresholds:
            threshold.Settings["ThresholdMode"] = xrfdc.TRSHD_OFF

[<xrfdc.RFdcAdcTile object at 0xffff7ee0f9d0>, <xrfdc.RFdcAdcTile object at 0xffff7ee9cf40>, <xrfdc.RFdcAdcTile object at 0xffff7ee9cfd0>, <xrfdc.RFdcAdcTile object at 0xffff7ee9d540>]


RuntimeError: Function XRFdc_GetThresholdSettings call failed
stdout: metal: error:     
 ADC 1 block 0 not available in XRFdc_GetThresholdSettings


In [4]:
#GPIO Switch Setup

switch = GPIO(GPIO.get_gpio_pin(0), "out")

def set_switch(val):
    """
    val = 0 sets A to -5 V, B to 0 V
    val = 1 set A to 0 V, B to -5 V
    """
    switch.write(val)

In [5]:
### Signal Source Generation 

# Create instances and start for the LMX2595
adl1 = ADL5960(ol.spi_adl_0)  
adl2 = ADL5960(ol.spi_adl_1) 

# Create and display the UI for signal generation
sig_ui = SignalSourceUI(mmio_spi_controller = ol.spi_lmx_0)
sig_ui.source.set_active()
source_layout = sig_ui.create_layout()

Initalized source
Initializing signal source with Start Frequency: 10000000 Stop Frequency: 20000000000 Resolution: 100.0


### Real Time Output Plot

In [6]:
fs = 147.456e6 # Sampling frequency
n = 65536 # Number of samples
T = n/fs

In [7]:
#Important Functions!
def read_dma():
    # Trigger the DMA transfer and wait for the result
    out_buffer = allocate(400024 * 4, dtype=np.int32)
    # Trigger the DMA transfer and wait for the result
    dma_interface.recvchannel.transfer(out_buffer)
    dma_interface.recvchannel.wait()
    return out_buffer

In [10]:
#Integrated time and s-parameter domain plot 

# Constants
PLOT_SIZE = 3e-6  # Time plot size
freq_list = sig_ui.source.generate_freq_points()  # Frequency points from signal source
freq_list_length = len(freq_list)

# Initialize storage for S-parameters
plot_S11_mag = np.zeros(freq_list_length)
plot_S11_phase = np.zeros(freq_list_length)
plot_S12_mag = np.zeros(freq_list_length)
plot_S12_phase = np.zeros(freq_list_length)
plot_S21_mag = np.zeros(freq_list_length)
plot_S21_phase = np.zeros(freq_list_length)
plot_S22_mag = np.zeros(freq_list_length)
plot_S22_phase = np.zeros(freq_list_length)

# Shared data for time-domain plots
time_data = np.linspace(0, T, n)  # Time data (X-axis)
time_x_axis = [time_data, time_data, time_data, time_data]
real0, imag0, real1, imag1, real2, imag2, real3, imag3 = (
    np.zeros(1000) for _ in range(8)
)

# Time-domain Plot
fig_time = plt.figure(title="Real-time Sensor Data", animation_duration=0)
time_lines = plt.plot([], [], 
                      colors=["blue", "red", "green", "orange"],
                     labels=["ADC0_Port1Reverse", "ADC1_Port2Reverse", "ADC2_Port1Forward", "ADC3_Port2Forward"])  # Time-domain lines
plt.xlim(0, PLOT_SIZE)
plt.ylim(2000, -2000)
plt.xlabel("Time [s]")
plt.legend()

# Frequency-domain Plot for Magnitude
fig_freq = plt.figure(title="S-Parameter Magnitude", animation_duration=0)
freq_lines = plt.plot(
    [], [],
    colors=["blue", "red", "green", "orange"],
    labels=["S11", "S12", "S21", "S22"]
)  # S-parameter magnitude lines
plt.xlim(freq_list[0], freq_list[-1])
plt.xlabel("Frequency [Hz]")
plt.legend()

# Toggle button for live updates
is_running = widgets.ToggleButton(
    value=False,
    description="Running",
    icon="play",
    tooltip="Start/Stop the live plot",
)

fir_filter = signal.firwin(15, [1e6 - 5e3, 1e6 + 5e3], fs = 147.456e6, pass_zero = False)

# Function to update plots
def update_plots():
    global real0, imag0, real1, imag1, real2, imag2, real3, imag3
    while is_running.value:
        freq_index = sig_ui.source.get_current_index()

        #Set switch
        set_switch(0); # 0 is Port 1 active
        
        # Read out DMA here
        out_buffer = read_dma()
        out_buffer_np = np.array(out_buffer, dtype=np.int32)

        # Split buffers and process data
        port1_reverse_buffer = out_buffer_np[0::4]
        port2_reverse_buffer = out_buffer_np[1::4]
        port1_forward_buffer = out_buffer_np[2::4]
        port2_forward_buffer = out_buffer_np[3::4]
        real_port2_forward, imag_port2_forward = VNAfunc.iq_break_data_np(port2_forward_buffer)
        real_port2_reverse, imag_port2_reverse = VNAfunc.iq_break_data_np(port2_reverse_buffer)
        real_port1_forward, imag_port1_forward = VNAfunc.iq_break_data_np(port1_forward_buffer)
        real_port1_reverse, imag_port1_reverse = VNAfunc.iq_break_data_np(port1_reverse_buffer)
        
        # Filter out LO spike.
        filtered_port2_forward = signal.convolve(real_port2_forward + 1j * imag_port2_forward, fir_filter, mode = "same")
        filtered_port2_reverse = signal.convolve(real_port2_reverse + 1j * imag_port2_reverse, fir_filter, mode = "same")
        filtered_port1_forward = signal.convolve(real_port1_forward + 1j * imag_port1_forward, fir_filter, mode = "same")
        filtered_port1_reverse = signal.convolve(real_port1_reverse + 1j * imag_port1_reverse, fir_filter, mode = "same")

        # Calculate S-parameters S11 and S12
        S11_mag, S11_phase = VNAfunc.calculate_S_param(filtered_port1_forward, filtered_port1_reverse) 
        S12_mag, S12_phase = VNAfunc.calculate_S_param(filtered_port1_forward, filtered_port2_forward) 
        plot_S11_mag[freq_index] = S11_mag
        plot_S12_mag[freq_index] = S12_mag
        
        #Set switch
        #set_switch(0);
        
        # Read out DMA 
#         out_buffer = read_dma()
#         out_buffer_np = np.array(out_buffer, dtype=np.int32)

#         # Split buffers and process data
#         port1_reverse_buffer = out_buffer_np[0::4]
#         port1_forward_buffer = out_buffer_np[1::4]
#         port2_reverse_buffer = out_buffer_np[2::4]
#         port2_forward_buffer = out_buffer_np[3::4]
#         real_port2_forward, imag_port2_forward = VNAfunc.iq_break_data_np(port2_forward_buffer)
#         real_port2_reverse, imag_port2_reverse = VNAfunc.iq_break_data_np(port2_reverse_buffer)
#         real_port1_forward, imag_port1_forward = VNAfunc.iq_break_data_np(port1_forward_buffer)
#         real_port1_reverse, imag_port1_reverse = VNAfunc.iq_break_data_np(port1_reverse_buffer)
        
#         # Filter out LO spike.
#         filtered_port2_forward = signal.convolve(real_port2_forward + 1j * imag_port2_forward, fir_filter, mode = "same")
#         filtered_port2_reverse = signal.convolve(real_port2_reverse + 1j * imag_port2_reverse, fir_filter, mode = "same")
#         filtered_port1_forward = signal.convolve(real_port1_forward + 1j * imag_port1_forward, fir_filter, mode = "same")
#         filtered_port1_reverse = signal.convolve(real_port1_reverse + 1j * imag_port1_reverse, fir_filter, mode = "same")

#         # Calculate S-parameters S21 and S22
        S21_mag, S21_phase = VNAfunc.calculate_S_param(filtered_port2_forward, filtered_port1_forward) #ADC0 ADC2
        S22_mag, S22_phase = VNAfunc.calculate_S_param(filtered_port2_forward, filtered_port2_reverse) #ADC0 ADC1
        plot_S21_mag[freq_index] = S21_mag
        plot_S22_mag[freq_index] = S22_mag

        # Update time-domain plot
        time_lines.x = time_x_axis
        time_lines.y = [filtered_port2_forward.real, filtered_port2_reverse.real, filtered_port1_forward.real, filtered_port1_reverse.real]

        # Update frequency-domain plot
        freq_lines.x = [freq_list, freq_list, freq_list, freq_list]
        freq_lines.y = [plot_S11_mag, plot_S12_mag,plot_S21_mag, plot_S22_mag]

        #time.sleep(0.1)  # Delay for live update

# Function to start the update thread
def start_plots(change):
    if is_running.value:
        thread = Thread(target=update_plots, daemon=True)
        thread.start()

# Observe toggle state for start/stop
is_running.observe(start_plots, names="value")

# Display the UI components
from ipywidgets import HBox, VBox, Layout

# Define layouts for the plots
plot_layout = Layout(width='90%', height='400px', border='1px solid black', margin='0 1%')

# Wrap each plot in a VBox with the specified layout
fig_time_widget = VBox([fig_time], layout=plot_layout)
fig_freq_widget = VBox([fig_freq], layout=plot_layout)

# Define the layout for the plots with stretching
plot_layout = widgets.VBox([is_running, fig_time_widget, fig_freq_widget], layout=widgets.Layout(width='100%', height='100%'))

In [11]:
# Wrap both source layout and plot layout into a horizontal layout (HBox)
main_layout = widgets.HBox([
    widgets.VBox([source_layout], layout=widgets.Layout(width='50%')),  # Source layout 50% width
    widgets.VBox([plot_layout], layout=widgets.Layout(width='50%'))  # Plot layout 50% width
])

display(main_layout)